In [7]:
import pandas as pd
import numpy as np
import gc
from typing import List

import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)


def get_train_data(TRAIN_PATH: str) -> pd.DataFrame:
    '''Returns train dataset'''
    df_train = pd.read_parquet(TRAIN_PATH)
    return df_train


def get_test_data(TEST_PATH: str) -> pd.DataFrame:
    '''Returns test dataset'''
    df_test = pd.read_parquet(TEST_PATH)
    return df_test


def get_target(TARGET_PATH: str) -> pd.DataFrame:
    '''Retruns dataset with train targets'''
    df_train_target = pd.read_csv(TARGET_PATH)
    return df_train_target


def get_train_data_with_target_merged(df_train: pd.DataFrame, df_train_target: pd.DataFrame) -> pd.DataFrame:
    '''Retruns train dataset with target variable merged'''
    df_train_w_target = (
        df_train
        .merge(df_train_target,
            on='customer_ID',
            how='left'
        )
    )
    # df_train_w_target.groupby('target', dropna=False).count()['customer_ID']
    '''
    target
    0    4153582
    1    1377869
    Name: customer_ID, dtype: int64    
    '''
    return df_train_w_target


def get_all_features(df: pd.DataFrame) -> List:
    '''Returns list of all features from the dataset'''
    return list(df)


def get_cat_features() -> List:
    '''Returns list of categorical features from the dataset'''
    cat_features = ['B_30', 'B_38', 'D_114', 
                    'D_116', 'D_117', 'D_120', 
                    'D_126', 'D_63', 'D_64', 
                    'D_66', 'D_68']
    
    return cat_features


def get_num_features(all_features: List, cat_features: List) -> List:
    '''Returns list of all numerical features from the dataset'''
    num_feats = [col for col in all_features if col not in cat_features + ['customer_ID', 'S_2', 'target']]

    return num_feats


def get_df_w_aggrs(df: pd.DataFrame, numerical_features: List) ->  pd.DataFrame:
    '''Returns dataframe with generated aggregates based on numerical features'''

    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer

    df_min = (df
        .groupby(cid)
        .min()[numerical_features]
        .rename(columns={f: f"{f}_min" for f in numerical_features})
    )
    print(df_min.shape)

    df_max = (df
        .groupby(cid)
        .max()[numerical_features]
        .rename(columns={f: f"{f}_max" for f in numerical_features})
    )
    print(df_max.shape)

    df_avg = (df
        .drop('S_2', axis='columns')
        .groupby(cid)
        .mean()[numerical_features]
        .rename(columns={f: f"{f}_avg" for f in numerical_features})
    )
    print(df_avg.shape)

    df_last = (df
        .loc[last, numerical_features]
        .rename(columns={f: f"{f}_last" for f in numerical_features})
        .set_index(np.asarray(cid[last]))
    )
    print(df_last.shape)

    df_aggrs = (pd.concat([df_min, df_max, df_avg, df_last], axis=1)
        .reset_index()
        .rename(columns={'index': 'customer_ID'})
    )
    print(df_aggrs.shape)

    '''
    del df, df_min, df_max, df_avg, cid, last
    gc.collect()
    '''
    return df_aggrs


def check_zapolnenie(df: pd.DataFrame) -> pd.DataFrame:
    '''Returns pd.DataFrame with isNotNullShare of each column of given df'''
    # Calculate percent of not null share each column 
    col_pct_notNull = [] 
    for col in df.columns: 
        percent_notNull = np.mean(~df[col].isnull())*100 
        col_pct_notNull.append([col, percent_notNull]) 
        
    col_pct_notNull_df = pd.DataFrame(col_pct_notNull, columns = ['column_name','isNotNullShare']).sort_values(by = 'isNotNullShare', ascending = False) 
    #print(col_pct_notNull_df)
    return col_pct_notNull_df

In [8]:
df_train = get_train_data(TRAIN_PATH='./data/train.parquet')

all_features = get_all_features(df_train)
cat_features = get_cat_features()
num_features = get_num_features(all_features, cat_features)
# len(all_features), len(cat_features), len(num_features) -> (190, 11, 178)

df_train_agg = get_df_w_aggrs(df=df_train, numerical_features=num_features)
df_train_target = get_target(TARGET_PATH='./data/train_labels.csv')
df_train = get_train_data_with_target_merged(df_train=df_train_agg, df_train_target=df_train_target)

'''
df_train.target.value_counts()
target
0    340085
1    118828
Name: count, dtype: int64    
'''

df_test = get_test_data(TEST_PATH='./data/test.parquet')
df_test = get_df_w_aggrs(df=df_test, numerical_features=num_features)

(458913, 177)
(458913, 177)
(458913, 177)
(458913, 177)
(458913, 709)
(924621, 177)
(924621, 177)
(924621, 177)
(924621, 177)
(924621, 709)


In [10]:
df_train.shape, df_test.shape 

((458913, 710), (924621, 709))

In [11]:
df_train

,customer_ID,P_2_min,D_39_min,B_1_min,B_2_min,R_1_min,S_3_min,D_41_min,B_3_min,D_42_min,...,D_137_last,D_138_last,D_139_last,D_140_last,D_141_last,D_142_last,D_143_last,D_144_last,D_145_last,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0,0.001930,1.000242,0.000263,0.098882,0.0,0.000783,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.002970,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.861109,0,0.006711,0.819772,0.001023,0.089799,0.0,0.000861,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003169,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.797670,0,0.001472,0.810796,0.003540,NaN,0.0,0.000626,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000834,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.567442,0,0.005910,0.812053,0.000199,0.149216,0.0,0.000053,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.005560,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.805045,0,0.000776,0.810670,0.000336,0.166190,0.0,0.000049,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.006944,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0.730505,0,0.006084,1.000524,0.000072,0.103766,0.0,0.000242,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.003009,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.831279,0,0.292360,0.030446,0.000569,0.194697,0.0,0.231867,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.009230,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.756983,0,0.012998,0.004150,0.000303,0.066648,0.0,0.000675,NaN,...,-1,-1,0,0,0.000000,NaN,0,0.000340,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.754129,0,0.000281,0.182818,0.000149,0.182067,0.0,0.039642,NaN,...,-1,-1,1,0,0.949723,0.446255,1,0.002502,2,1


In [9]:
df_test

,customer_ID,P_2_min,D_39_min,B_1_min,B_2_min,R_1_min,S_3_min,D_41_min,B_3_min,D_42_min,...,D_136_last,D_137_last,D_138_last,D_139_last,D_140_last,D_141_last,D_142_last,D_143_last,D_144_last,D_145_last
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.568930,0,0.007959,0.810456,0.0,0.149413,0.000000,0.002347,0.103745,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003690,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.794469,0,0.014187,1.000782,0.0,0.055804,0.000000,0.001597,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.000247,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.673112,0,0.001483,0.810072,0.0,0.132608,0.000000,0.000155,NaN,...,-1,-1,-1,1,0,0.896224,0.150203,1,0.457819,1
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.428457,7,0.149511,0.032696,0.0,0.181200,0.000000,0.389167,NaN,...,-1,-1,-1,1,0,0.919774,0.255263,1,0.500924,2
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.254478,1,0.357828,0.020167,0.0,0.155480,0.000000,0.350645,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.001558,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.597667,0,0.001874,0.810936,0.0,0.102306,0.000000,0.020869,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.003278,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.471303,0,0.015243,0.082507,0.0,0.400732,0.000000,0.007571,0.146153,...,-1,-1,-1,0,0,0.000000,NaN,0,0.005432,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.195671,0,0.013811,0.171140,0.0,0.158748,0.077758,0.002672,0.514219,...,-1,-1,-1,0,0,0.000000,NaN,0,0.004849,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.538731,0,0.049776,0.103310,0.0,0.179635,0.000000,0.099891,NaN,...,-1,-1,-1,0,0,0.000000,NaN,0,0.007482,0
